# Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# CNN

|||
|-|-|
|![](https://wikidocs.net/images/page/80437/sentence_matrix.PNG)|![](https://wikidocs.net/images/page/80437/conv1d.PNG)|

<br>

이미지나 영상 데이터를 처리할 때 사용하는 convolution을 자연어에 적용. <br>

<br>

<font style="font-size=20px"> 특징 </font>

병렬 처리: 입력 데이터의 모든 위치에서 필터를 동시에 적용할 수 있어, 각 시간 단계에 대해 순차적으로 계산해야 하는 RNN에 비해 병렬 처리 속도가 훨씬 빠름 <br>

국소적 특징 추출: convolution 연산을 통해 입력 데이터에서 국소적인 패턴이나 특징을 효과적으로 추출. <br>
-> 단어나 구의 의미를 파악하는 데 유리. <br>

과적합 방지: filter의 수와 크기를 조정함으로써 과적합을 줄이는 데 도움을 줄 수 있음. <br>
-> RNN보다 더 적은 수의 매개변수로도 높은 성능을 유지할 수 있는 가능성을 제공. <br>

Scalability: 다양한 크기의 filter를 적용하여 다중 해상도에서 특징을 학습할 수 있음. <br>
-> 문맥의 다양한 수준에서 패턴을 잡아낼 수 있음.

### Layers

#### Convolution

<img src="https://wikidocs.net/images/page/80437/%EB%84%A4%EB%B2%88%EC%A7%B8%EC%8A%A4%ED%85%9D.PNG" width="600">

filter를 이용해 이미지를 스캔하면서 특징을 추출. <br>
filter는 학습 과정에서 업데이트. <br>
다양한 필터를 통해 문장 내 주요 정보 파악. <br>

<br>
<br>

<font style="font-size:20px"> 사용 방법 </font>

> ```python
> nn.Conv1d(
>     in_channels,     # (int) 입력 채널 수. seq_len 또는 이전 out_channels의 수와 동일
>     out_channels,    # (int) 출력 채널 수. filter의 개수. 생성될 출력 텐서의 채널 수.
>     kernel_size,     # (int or tuple) filter의 크기. 몇 개 단위의 토큰 내 정보를 파악할 것인지 결정.
>     stride=1,        # (int or tuple) 스트라이드. 필터를 적용할 때 이동하는 간격.
>     padding='same',  # (int, tuple, or str) 패딩. 'same'으로 설정하면 입력과 동일한 크기의 출력을 얻도록 패딩을 자동으로 추가. 'valid'는 padding을 사용하지 않음.
> )
> ```

In [7]:
x = torch.randint(0, 20, (32, 20))
x.shape # batch_size, seq_len

torch.Size([32, 20])

In [8]:
x= nn.Embedding(20, 128)(x)
x.shape # batch_size, seq_len, dim

torch.Size([32, 20, 128])

In [19]:
conv_out = nn.Conv1d(
    20,
    64,
    kernel_size=2,
    padding='same'
)(x)

#### Pooling

<img src="https://wikidocs.net/images/page/80437/%EB%A7%A5%EC%8A%A4%ED%92%80%EB%A7%81.PNG" width="400">

특성을 요약하여 데이터 크기를 줄이고 연산 효율을 높입. <br>
convolution의 연산 결과에 적용. <br>

<br>

Max Pooling: 가장 두드러지는 특징을 찾음. <br>

Average Pooling: 평균적은 특징을 찾음. <br>

In [20]:
conv_out.shape

torch.Size([32, 64, 128])

In [27]:
max_out = nn.MaxPool1d(4)(conv_out)

In [22]:
conv_out = nn.Conv1d(
    20,
    20,
    kernel_size=2,
    padding='same'
)(x)

In [23]:
(x + conv_out).shape

torch.Size([32, 20, 128])

#### Fully Connected Layer

최종 예측을 위한 layer. <br>
이전 층에서 추출한 특징을 바탕으로 텍스트가 어떤 클래스에 속하는지 예측. <br>
전통적인 다층 퍼셉트론처럼 동작하며, CNN의 분류 결과를 출력. <br>

In [28]:
flatten = max_out.flatten(start_dim=1)

In [29]:
nn.Linear(640, 2)(flatten)

tensor([[-4.0051e-01, -4.1858e-01],
        [ 1.3573e-01, -5.2509e-01],
        [-2.5074e-02, -6.0886e-01],
        [-9.8061e-02, -5.8438e-01],
        [ 1.2339e-01, -4.9635e-01],
        [ 1.4166e-01, -1.0940e+00],
        [-2.4857e-01, -2.1598e-01],
        [-3.5043e-02, -5.5456e-01],
        [-6.5026e-02, -3.3361e-01],
        [ 2.1597e-01, -2.6495e-01],
        [ 2.8623e-01, -2.2976e-01],
        [ 4.3730e-01, -1.0205e-01],
        [-3.3234e-01, -4.5240e-01],
        [ 8.2908e-02, -5.2466e-01],
        [ 4.2935e-01, -3.5379e-01],
        [-7.8750e-02, -2.2148e-01],
        [ 2.6232e-01, -4.4683e-02],
        [ 3.3722e-01, -5.6358e-01],
        [ 2.5005e-01, -5.0370e-01],
        [ 7.8430e-01, -3.4337e-01],
        [-1.1817e-02, -4.5781e-01],
        [ 3.6433e-02, -3.6756e-01],
        [ 1.1437e-01, -1.7608e-01],
        [-7.4911e-02, -8.1453e-01],
        [ 2.0693e-01, -4.4502e-02],
        [ 1.6072e-01, -4.2147e-01],
        [ 1.1877e-02, -6.8338e-01],
        [ 4.0276e-01, -3.548

In [3]:
!pip install -U datasets

In [2]:
from datasets import load_dataset

In [16]:
train_dataset = load_dataset('stanfordnlp/imdb', split='train')
test_dataset = load_dataset('stanfordnlp/imdb', split='test')


In [27]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [37]:
train_dataset[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [11]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
import re

In [ ]:
word_to_idx = dict(zip(words, range(len(words))))
idx_to_word = dict(zip(range(len(words)), words))

In [12]:
def preprocessisng(sample):
    text = re.sub('([,?!\'",\[\]\(\)\{\}])', ' \\1', sample['text'])
    text = text.lower()
    words = text.split()
    ids = [word_to_idx.get(word) for word in words]

    return {'text': ids}

KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['test', 'train', 'unsupervised']"

In [ ]:
# IMDB datasets
imdb = load_dataset('imdb')

train, valid = imdb.train_test_split(
    train,
    test_size=0.2
    )

train_text = ''.join(train['text'])
test_text = ''.join(test['text'])

train_text = re.sub('([,?!\'",\[\]\(\)\{\}])', ' \\1', train_text)
test_text = re.sub('([,?!\'",\[\]\(\)\{\}])', ' \\1', train_text)

total_text = ' '.join([train_text, test_text])
total_text = total_text.lower()

words = total_text.split()
words = set(words)
words = list(words)

word_to_idx = dict(zip(words, range(len(words))))
idx_to_word = dict(zip(range(len(words)), words))

def preprocessisng(sample):
    text = re.sub('([,?!\'",\[\]\(\)\{\}])', ' \\1', sample['text'])
    text = text.lower()
    words = text.split()
    ids = [word_to_idx.get(word) for word in words]

    return {'text': ids}

## Practice

### Multiple CNN Filters

<img src="https://figures.semanticscholar.org/7c35b1ccefedcadefa9670deccc0bf651b996666/9-Figure3-1.png" width=600>

<br>

다양한 CNN filter를 결합하여 예측 <br>

In [11]:
conv_out_with_3 = nn.Conv1d(
    20,
    64,
    kernel_size=3,
    padding='same'
)(x)
print(conv_out_with_3.shape)

conv_out_with_5 = nn.Conv1d(
    20,
    64,
    kernel_size=5,
    padding='same'
)(x)
print(conv_out_with_5.shape)

torch.Size([32, 64, 128])
torch.Size([32, 64, 128])


In [12]:
concat = torch.cat([conv_out_with_3, conv_out_with_5], axis=1)

In [14]:
max_out_with_3 = nn.MaxPool1d(kernel_size=2)(conv_out_with_3)
max_out_with_5 = nn.MaxPool1d(kernel_size=2)(conv_out_with_5)

In [15]:
torch.cat([max_out_with_3, max_out_with_5], axis=1).shape

torch.Size([32, 128, 64])

In [17]:
max_out_with_3 = nn.MaxPool1d(kernel_size=128)(conv_out_with_3)
max_out_with_5 = nn.MaxPool1d(kernel_size=128)(conv_out_with_5)

In [18]:
torch.cat([max_out_with_3, max_out_with_5], axis=-1).shape

torch.Size([32, 64, 2])

### CNN-LSTM

<img src="https://konukoii.com/blog/wp-content/uploads/2018/02/cnn-lstm-konukoii.png" width=600>

<br>

CNN으로 feature extraction 이후 LSTM을 통해 학습.

### LSTM-CNN

<img src="https://konukoii.com/blog/wp-content/uploads/2018/02/lstm-cnn-konukoii.png" width=600>

<br>

LSTM Layer로 feature extraction 이후 CNN layer로 feature extraction

### CNN + LSTM

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S1532046420301672-gr4.jpg" width=600>

<br>

각 CNN과 LSTM 결과물을 concat하여 활용